In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv

#import deeplabcut


Autosaving every 180 seconds


In [41]:
data= np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/traces_means.npz')
tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']
# print (tracesx.shape)
# t = np.arange(tracesx.shape[0])/25.

# if False:
#     fig=plt.figure()
#     ax=plt.subplot(2,1,1)
#     plt.plot(t,tracesx)
#     ax=plt.subplot(2,1,2)
#     plt.plot(t,tracesy)
#     plt.show()

In [42]:
# where_are_NaNs = np.isnan(tracesx)
# tracesx[where_are_NaNs] = 9999


# where_are_NaNs = np.isnan(tracesy)
# tracesy[where_are_NaNs] = 9999


In [53]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

global current_index, selected_animal

# reload data
tracesx = data['tracesx_mean'].copy()
tracesy = data['tracesy_mean'].copy()


# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250
history = np.ones((4,5),'int32')+100

# save current frame of video
current_index = []
current_index.append(0)

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)

# initialize start and ends of video to be analyzed
video_name = "/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi"
frame_rate = 25
original_vid = cv2.VideoCapture(video_name)

# initialize video
loc = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
ret, frame = original_vid.read()
n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize scatter plots
ax_video = plt.subplot(gs[:, 1:4])
img = ax_video.imshow(frame)
ax_video.set_xlim(150,1200)
ax_video.set_ylim(50,800)

axes_animals=[]
for k in range(4):
    x_loc = tracesx[0,k]
    y_loc = tracesy[0,k]
    if np.isnan(x_loc) or np.isnan(y_loc):
        x_loc=100
        y_loc=100
    axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

# plot traces
ax_tracesx = plt.subplot(gs[:2, :1])
ax_tracesx.set_title("DLC-x")
init_length = 10*25
t = np.arange(init_length)/25.
ax1 = []
for k in range(4):
    temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
    ax1.append(temp)

ax_tracesx.set_xticks([])
ax_tracesx.set_ylim(0,1280)

# plot y traces
ax_tracesy = plt.subplot(gs[2:, :1])
ax_tracesy.set_title("DLC-y")
ax2 = []
for k in range(4):
    temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
    ax2.append(temp)

ax_tracesy.set_xlabel("Time (sec)")
ax_tracesy.set_ylim(0,1024)
# add 
#ax.margins(x=0)

##location of sliders
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
#sfreq = Slider(axfreq, 'Frame',0.20, 18, valinit=0, dragging=True,  fc='blue')    
#sfreq.val=val


# update the slider
def update(val):
    #amp = samp.val
    index = int(sfreq.val)
    current_index[0] = index
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
    fig.canvas.draw_idle()

sfreq.on_changed(update)
#samp.on_changed(update)

# resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
# button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

# def reset(event):
#     sfreq.reset()
#     #samp.reset()
# button.on_clicked(reset)

# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.8, 0.07, 0.1], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
    
def animal_select(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    
    
animal_buttons.on_clicked(animal_select)

# ANIMAL SWITCHER
rax2 = plt.axes([0.01, 0.65, 0.07, 0.1], facecolor=axcolor)
animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax2.set_title('Switch')

def animal_switch_function(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k
    print ("switching to : ", labels[switch_id])
    print ("current index inside switch:", current_index)

    # search active trace snippet
    # search forward

    def find_snippet(current_index, tracesx, id_):
        for k in range(current_index[0]+1, n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break
        if k==(n_frames-1):
            end = n_frames-1

        # search backward
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
            #if tracesx[k, current_index[0]]==9999:
                start = k
                break

        if k==(0):
            start = 0
    
        return end, start
    
    end2, start2 = find_snippet(current_index, tracesx, switch_id)
            
    print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    
    end1, start1 = find_snippet(current_index, tracesx, selected_animal[0])
    
    print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
        
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()

    # replace switch id info
    tracesx[start1:end1, switch_id] = tracesx[start1:end1, selected_animal[0]]
    tracesy[start1:end1, switch_id] = tracesy[start1:end1, selected_animal[0]]
    
    # replace currental animal info
    tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id]
    tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id]
  
    # update plots
    update(current_index[0])
    
animal_switch.on_clicked(animal_switch_function)


# search for the next continuous chunk of red
def next_(event):
    
    # 
    for k in range(current_index[0], n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)
    
    for k in range(start_nan, n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            end_nan=k
            break

    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = end_nan
    sfreq.val = current_index[0]
    fig.canvas.draw_idle()

    print ("..... to : ", current_index[0])
    update(current_index[0])

    
    # search for the next continuous chunk of red
    
def previous(event):
    
    # 
    start_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)

    end_nan=0
    for k in range(start_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            end_nan=k
            break

    start_data =0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k
            break
            
    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = start_data+1
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    fig.canvas.draw_idle()

    
def save(event):
    
    np.savez('/home/cat/traces_fixed', 
             tracesx = tracesx,
             tracesy = tracesy)
# def prev(event):
#     self.ind -= 1
#     i = self.ind % len(freqs)
#     ydata = np.sin(2*np.pi*freqs[i]*t)
#     l.set_ydata(ydata)
#     plt.draw()

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.01, 0.4, 0.04, 0.04])
bnext = Button(axnext, 'Next')
bnext.on_clicked(next_)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.01, 0.3, 0.04, 0.04])
bprev = Button(axprev, 'Previous')
bprev.on_clicked(previous)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.01, 0.2, 0.04, 0.04])
bsave = Button(axsave, 'Save')
bsave.on_clicked(save)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)


plt.show()

current indx updated: [290]
current indx updated: [376]
current indx updated: [699]
current indx updated: [1175]
current indx updated: [1536]
current indx updated: [1708]
current indx updated: [1822]
searching... start_nan:  2192
Fast forwarding from:  1822
..... to :  2195
current indx updated: [2195]
searching... start_nan:  2236
Fast forwarding from:  2195
..... to :  2239
current indx updated: [2239]
searching... start_nan:  2275
Fast forwarding from:  2239
..... to :  2283
current indx updated: [2283]
searching... start_nan:  2282
Fast forwarding from:  2283
..... to :  2239
current indx updated: [2239]
searching... start_nan:  2238
Fast forwarding from:  2239
..... to :  2195
current indx updated: [2195]
searching... start_nan:  2236
Fast forwarding from:  2195
..... to :  2239
current indx updated: [2239]
switching to :  pup1 (cyan)
current index inside switch: [2239]
Switch animal id:  2 , start/end:  2238 2244
selected animal id:  0 , start/end:  2238 2275
current indx updated

searching... start_nan:  1147
Fast forwarding from:  1148
..... to :  1111
current indx updated: [1111]
searching... start_nan:  1110
Fast forwarding from:  1111
..... to :  1081
current indx updated: [1081]
searching... start_nan:  1108
Fast forwarding from:  1081
..... to :  1111
current indx updated: [1111]
searching... start_nan:  1144
Fast forwarding from:  1111
..... to :  1148
current indx updated: [1148]
searching... start_nan:  1176
Fast forwarding from:  1148
..... to :  1179
current indx updated: [1179]
searching... start_nan:  1250
Fast forwarding from:  1179
..... to :  1253
current indx updated: [1253]
searching... start_nan:  1750
Fast forwarding from:  1253
..... to :  1753
current indx updated: [1753]
searching... start_nan:  1761
Fast forwarding from:  1753
..... to :  1769
current indx updated: [1769]
searching... start_nan:  1773
Fast forwarding from:  1769
..... to :  1776
current indx updated: [1776]
searching... start_nan:  1784
Fast forwarding from:  1776
..... 

searching... start_nan:  4084
Fast forwarding from:  3206
..... to :  4087
current indx updated: [4087]
searching... start_nan:  4094
Fast forwarding from:  4087
..... to :  4102
current indx updated: [4102]
searching... start_nan:  4176
Fast forwarding from:  4102
..... to :  4180
current indx updated: [4180]
searching... start_nan:  4275
Fast forwarding from:  4180
..... to :  4282
current indx updated: [4282]
searching... start_nan:  4338
Fast forwarding from:  4282
..... to :  4342
current indx updated: [4342]
searching... start_nan:  4344
Fast forwarding from:  4342
..... to :  4346
current indx updated: [4346]
searching... start_nan:  4349
Fast forwarding from:  4346
..... to :  4351
current indx updated: [4351]
searching... start_nan:  4371
Fast forwarding from:  4351
..... to :  4373
current indx updated: [4373]
searching... start_nan:  4495
Fast forwarding from:  4373
..... to :  4539
current indx updated: [4539]
switching to :  pup2 (green)
current index inside switch: [4539]

In [36]:
data1 = np.load('/home/cat/traces_fixed.npy.npz')
print (data1['tracesx'])

[[ 324.056        nan       nan       nan]
 [ 324.253        nan       nan       nan]
 [ 324.365        nan       nan       nan]
 ...
 [ 221.123   859.061  1179.818   275.199 ]
 [ 234.9225  859.061  1179.71    280.644 ]
 [ 219.568   859.061  1179.6125  278.984 ]]
